# Neighborhood Recommendations for Transfer Employees Through K-Means Clustering

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#01">01. Import the necessary python libraries</a></li>
        <li><a href="#02">02. Get London Neighborhoods and their Respective Latitudes and Longitudes</a></li>
        <li><a href="#03">03. Get Frankfurt Neighborhoods and their Respective Latitudes and Longitudes</a></li>
        <li><a href="#04">04. Get New York City Neighborhoods and their respective latitudes and longitudes</a></li>
        <li><a href="#05">05. Combine Neighborhood Geocodes from All Three Cities</a></li>
        <li><a href="#06">06. Explore London's Neighborhood </a></li>
        <li><a href="#07">07. Explore Frankfurt's Neighborhood</a></li>
        <li><a href="#08">08. Explore New Yorks's Neighborhood</a></li>
        <li><a href="#09">09. Using Foursquare API Get Nearby Venues</a></li>
        <li><a href="#10">10. Cluster Neighborhoods</a></li>
        <li><a href="#11">11. Examine Clusters</a></li>
        <li><a href="#12">12. Sample Use Case</a></li>
    </ul>
</div>

<hr>

### 1. Import the necessary python libraries <a name="01"></a>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# !conda install -c conda-forge wikipedia --yes
import wikipedia as wp # library to read wikipedia pages.

print('Libraries imported.')

Libraries imported.


#### Set the width of the notebook to 98% for easy viewing

In [3]:
# Set up this notebook so that it's easy to work with.
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 98%; }
    div#menubar-container     { width: 75%; }
    div#maintoolbar-container { width: 99%; }   
</style>
""")
)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import os
path="D:\Course\Applied.Data.Science.Capstone"
os.chdir(path)

### 2. Get London Neighborhoods and the Respective Latitudes and Longitudes <a name="02"></a>

In [9]:
# Get wikipedia dataset for London
# https://en.wikipedia.org/wiki/List_of_areas_of_London
London_html = wp.page("List_of_areas_of_London").html().encode("UTF-16")

# Get the table containing the 46 city districts of Frankfurt on the Wikipedia page
df_London = pd.read_html(London_html, header = 0)[1][['Location']]

# Rename column name from location to Neighborhood.
df_London.rename(columns={'Location':'Neighborhood'}, inplace=True)

# Create a new column called City and assign london to all rows
df_London = df_London.assign(City = 'London')

# Remove words within parentheses, along with the parentheses
df_London['Neighborhood'] = df_London['Neighborhood'].str.replace(r"\(.*\)","")

# Get the latitude and Longitude of London Neighborhoods
geo_London = Nominatim(user_agent="Lon_Explorer")
df_London['Latitude'] = df_London['Neighborhood'].apply(geo_London.geocode).apply(lambda x: (x.latitude))
df_London['Longitude'] = df_London['Neighborhood'].apply(geo_London.geocode).apply(lambda x: (x.longitude))

# To avoid "too many request error" on geopy library, save the above created dataframe, remove duplicates and import back the cleanup list of neighborhoods.
df_London = pd.read_excel('London_Neighborhoods_Geocodes.xlsx')

# Rearrange columns for ease of use
df_London = df_London[['City', 'Neighborhood', 'Latitude', 'Longitude']]

# Print basic info about dataframe
print(df_London.shape)
df_London.head()

(525, 4)


,City,Neighborhood,Latitude,Longitude
0,London,Abbey Wood,51.487621,0.114050
1,London,Acton,51.508140,-0.273261
2,London,Addington,47.725036,-66.765785
3,London,Addiscombe,51.379692,-0.074282
4,London,Albany Park,41.971937,-87.716174


### 3. Get Frankfurt Neighborhoods and their respective latitudes and longitudes <a name="03"></a>

In [11]:
# Get wikipedia dataset for Frankfurt
# https://en.wikipedia.org/wiki/Frankfurt
Frankfurt_html = wp.page("Frankfurt").html().encode("UTF-16")

# Get the table containing the 46 city districts of Frankfurt on the Wikipedia page
df_Frankfurt = pd.read_html(Frankfurt_html, header = 0)[4][['City district (Stadtteil)']] # Table titled: "Population of the 46 city districts on 31 December 2009"

# Drop the last row, because it has summary information of Frankfurt.
df_Frankfurt.drop(df_Frankfurt.tail(1).index,inplace=True) # drop last n rows

# Rename column name from location to Neighborhood.
df_Frankfurt.rename(columns={'City district (Stadtteil)':'Neighborhood'}, inplace=True)

# Create a new column called City and assign london to all rows
df_Frankfurt = df_Frankfurt.assign(City = 'Frankfurt')

# Get the latitude and Longitude of Frankfurt Neighborhoods
geo_Frank = Nominatim(user_agent="Frankfurt_Explorer")
df_Frankfurt['Latitude'] = df_Frankfurt['Neighborhood'].apply(geo_Frank.geocode).apply(lambda x: (x.latitude))
df_Frankfurt['Longitude'] = df_Frankfurt['Neighborhood'].apply(geo_Frank.geocode).apply(lambda x: (x.longitude))

# Rearrange columns for ease of use
df_Frankfurt = df_Frankfurt[['City', 'Neighborhood', 'Latitude', 'Longitude']]

# Print basic info about dataframe
print(df_Frankfurt.shape)
df_Frankfurt.head()

(46, 4)


,City,Neighborhood,Latitude,Longitude
0,Frankfurt,Altstadt,31.504619,34.464127
1,Frankfurt,Innenstadt,50.112878,8.674922
2,Frankfurt,Bahnhofsviertel,50.107741,8.668736
3,Frankfurt,Westend-Süd,50.115245,8.662270
4,Frankfurt,Westend-Nord,50.126356,8.667921


### 4. Get New York City Neighborhoods and their respective latitudes and longitudes <a name="04"></a>

In [12]:
# We'll reuse the New York Neighborhoods data that we prepared in Week 3 of the class
df_Newyork = pd.read_excel('NYC_Neighborhoods.xlsx', usecols="B:D")

# Create a new column called City and assign london to all rows
df_Newyork = df_Newyork.assign(City = 'New York')

# Rearrange columns for ease of use
df_Newyork = df_Newyork[['City', 'Neighborhood', 'Latitude', 'Longitude']]

# Print basic info about dataframe
print(df_Newyork.shape)
df_Newyork.head()

(306, 4)


,City,Neighborhood,Latitude,Longitude
0,New York,Wakefield,40.894705,-73.847201
1,New York,Co-op City,40.874294,-73.829939
2,New York,Eastchester,40.887556,-73.827806
3,New York,Fieldston,40.895437,-73.905643
4,New York,Riverdale,40.890834,-73.912585


### 5. Combine Neighborhood Geocodes from All Three Cities <a name="05"></a>

In [13]:
# Combine all neighborhoods from London, Frankfurt, and New York to do a combined K-means cluster analysis
df_combined = pd.concat([df_London, df_Frankfurt, df_Newyork], ignore_index=True)

# Print basic info about dataframe
print(df_combined.shape)
df_combined.head()

(877, 4)


,City,Neighborhood,Latitude,Longitude
0,London,Abbey Wood,51.487621,0.114050
1,London,Acton,51.508140,-0.273261
2,London,Addington,47.725036,-66.765785
3,London,Addiscombe,51.379692,-0.074282
4,London,Albany Park,41.971937,-87.716174


### 6. Explore London's Neighborhood <a name="06"></a>

In [40]:
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="London_Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(latitude, longitude))

# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, city in zip(df_London['Latitude'], df_London['Longitude'], df_London['Neighborhood']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

The geograpical coordinates of London are 51.5073219, -0.1276474.


### 7. Explore Frankfurt's Neighborhood <a name="07"></a>

In [43]:
address = 'Frankfurt, Germany'

geolocator = Nominatim(user_agent="Frankfurt_Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Frankfurt are {}, {}.'.format(latitude, longitude))

# create map of Frankfurt using latitude and longitude values
map_frankfurt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, city in zip(df_Frankfurt['Latitude'], df_Frankfurt['Longitude'], df_Frankfurt['Neighborhood']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_frankfurt)  
    
map_frankfurt

The geograpical coordinates of Frankfurt are 50.1106444, 8.6820917.


### 8. Explore New Yorks's Neighborhood <a name="08"></a>

In [45]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="NY_Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of New York City are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_Newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, city in zip(df_Newyork['Latitude'], df_Newyork['Longitude'], df_Newyork['Neighborhood']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Newyork)  
    
map_Newyork

The geograpical coordinates of New York City are 40.7127281, -74.0060152.


In [17]:
# @hidden
CLIENT_ID = 'NEQM0YQY35OBBVOBPETJBLMBYQQZZKXW2CNA1AFVQTCXJLO0' # your Foursquare ID
CLIENT_SECRET = '0MQNFC43KVAHSDZQ2JZDRGZ5WIR4YJE4ZGLWEP22UFFXSCH1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NEQM0YQY35OBBVOBPETJBLMBYQQZZKXW2CNA1AFVQTCXJLO0
CLIENT_SECRET:0MQNFC43KVAHSDZQ2JZDRGZ5WIR4YJE4ZGLWEP22UFFXSCH1


### 9. Using Foursquare API Get Nearby Venues <a name="09"></a>

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
combined_venues = getNearbyVenues(names=df_combined['Neighborhood'],
                                   latitudes=df_combined['Latitude'],
                                   longitudes=df_combined['Longitude']
                                  )
print(combined_venues.shape)

Abbey Wood
Acton
Addington
Addiscombe
Albany Park
Aldborough
Aldgate
Aldwych
Alperton
Anerley
Angel
Aperfield
Archway
Ardleigh Green
Arkley
Arnos Grove
Balham
Bankside
Barbican
Barking
Barkingside
Barnehurst
Barnes
Barnes Cray
Barnet Gate
Barnet 
Barnsbury
Battersea
Bayswater
Beckenham
Beckton
Becontree
Becontree Heath
Beddington
Bedford Park
Belgravia
Bellingham
Belmont
Belsize Park
Belvedere
Bermondsey
Berrylands
Bethnal Green
Bexley 
Bexleyheath 
Bickley
Biggin Hill
Blackfen
Blackfriars
Blackheath
Blackwall
Blendon
Bloomsbury
Botany Bay
Bounds Green
Bow
Bowes Park
Brentford
Brent Cross
Brent Park
Brimsdown
Brixton
Brockley
Bromley
Bromley 
Bromley Common
Brompton
Brondesbury
Brunswick Park
Bulls Cross
Burnt Oak
Burroughs, The
Camberwell
Cambridge Heath
Camden Town
Canary Wharf
Cann Hall
Canning Town
Canonbury
Carshalton
Castelnau
Castle Green
Catford
Chadwell Heath
Chalk Farm
Charing Cross
Charlton
Chase Cross
Cheam
Chelsea
Chelsfield
Chessington
Childs Hill
Chinatown
Chinbrook
Chin

Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Ditmas Park
Wingate
Rugby
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charlesto

In [21]:
# Peek into the combined venues dataframe
combined_venues.sample(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17592,Battery Park City,40.711932,-74.016869,Vino e Grano,40.709953,-74.011574,Italian Restaurant
3837,Fitzrovia,51.518764,-0.141002,Salt Yard,51.519180,-0.136458,Tapas Restaurant
14522,Downtown,40.690844,-73.983463,Century 21 Department Store,40.690130,-73.983317,Department Store
21592,Blissville,40.737251,-73.932442,MTA - B24 Bus Stop (Van Dam),40.735161,-73.937458,Bus Station
16740,Greenwich Village,40.726933,-73.999914,Comedy Cellar,40.730130,-74.000402,Comedy Club


#### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(combined_venues['Venue Category'].unique())))

There are 520 uniques categories.


In [23]:
# one hot encoding
combined_onehot = pd.get_dummies(combined_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
combined_onehot['Neighborhood'] = combined_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [combined_onehot.columns[-1]] + list(combined_onehot.columns[:-1])
combined_onehot = combined_onehot[fixed_columns]

combined_onehot.head(3)

,Zoo Exhibit,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Apple Wine Pub,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Chaat Place,Champagne Bar,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Cigkofte Place,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Football Field,College Gym,College Quad,College Science Building,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Currywurst Joint,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,...,Kurdish Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscella

In [24]:
combined_onehot.shape

(22169, 520)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
combined_grouped = combined_onehot.groupby('Neighborhood').mean().reset_index()
print(combined_grouped.shape)
combined_grouped.sample(5)

(815, 520)


,Neighborhood,Zoo Exhibit,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Apple Wine Pub,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Chaat Place,Champagne Bar,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Cigkofte Place,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Football Field,College Gym,College Quad,College Science Building,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Currywurst Joint,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,...,Kosher Restaurant,Kurdish Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,M

#### First, let's write a function to sort the venues in descending order.

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = combined_grouped['Neighborhood']

for ind in np.arange(combined_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(combined_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,Playground,Wine Shop,Grocery Store,Campground,Yoshoku Restaurant,Eye Doctor,Exhibit,Event Space,Event Service,Ethiopian Restaurant
1,Acton,Pub,Gym / Fitness Center,Hotel,Fast Food Restaurant,Grocery Store,Shopping Mall,Thai Restaurant,Chinese Restaurant,Turkish Restaurant,Athletics & Sports
2,Addiscombe,Park,Bakery,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Café,Event Service,Ethiopian Restaurant,Farmers Market,Exhibit
3,Albany Park,Sandwich Place,Fried Chicken Joint,Chinese Restaurant,Diner,Café,Gas Station,Korean Restaurant,Cocktail Bar,Grocery Store,Pizza Place
4,Aldborough,Flower Shop,Construction & Landscaping,Farm,Yoshoku Restaurant,Falafel Restaurant,Factory,Eye Doctor,Exhibit,Event Space,Event Service


### 10. Cluster Neighborhoods <a name="10"></a>

Run *k*-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

combined_grouped_clustering = combined_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(combined_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 4, 3, 2, 2, 2, 3, 3, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

combined_merged = df_combined

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
combined_merged = combined_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

combined_merged.head() # check the last columns!

,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,London,Abbey Wood,51.487621,0.114050,0.0,Playground,Wine Shop,Grocery Store,Campground,Yoshoku Restaurant,Eye Doctor,Exhibit,Event Space,Event Service,Ethiopian Restaurant
1,London,Acton,51.508140,-0.273261,2.0,Pub,Gym / Fitness Center,Hotel,Fast Food Restaurant,Grocery Store,Shopping Mall,Thai Restaurant,Chinese Restaurant,Turkish Restaurant,Athletics & Sports
2,London,Addington,47.725036,-66.765785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,London,Addiscombe,51.379692,-0.074282,4.0,Park,Bakery,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Café,Event Service,Ethiopian Restaurant,Farmers Market,Exhibit
4,London,Albany Park,41.971937,-87.716174,3.0,Sandwich Place,Fried Chicken Joint,Chinese Restaurant,Diner,Café,Gas Station,Korean Restaurant,Cocktail Bar,Grocery Store,Pizza Place


### 11. Examine Clusters <a name="11"></a>

In [55]:
combined_merged[combined_merged['Cluster Labels']==2.0].sample(10)

,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
175,London,Fortis Green,51.590997,-0.153421,2.0,Indian Restaurant,Italian Restaurant,Mediterranean Restaurant,Deli / Bodega,Wine Shop,Gym / Fitness Center,Café,Thai Restaurant,Pub,Furniture / Home Store
437,London,Strawberry Hill,51.439168,-0.339301,2.0,Convenience Store,Train Station,Thai Restaurant,Wine Shop,Café,Pub,History Museum,Event Space,Eye Doctor,Exhibit
668,New York,South Side,40.710861,-73.958001,2.0,Bar,Coffee Shop,American Restaurant,Pizza Place,Breakfast Spot,Chinese Restaurant,Yoga Studio,Cocktail Bar,South American Restaurant,Mexican Restaurant
99,London,Clapham,51.462292,-0.138856,2.0,Pub,Café,Bar,Burger Joint,Grocery Store,Coffee Shop,Restaurant,Italian Restaurant,Cocktail Bar,Gastropub
519,London,Woodside Park,37.309857,-80.036707,2.0,Brewery,Yoshoku Restaurant,Fast Food Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
555,Frankfurt,Berkersheim,50.171166,8.701191,2.0,German Restaurant,Platform,Yoshoku Restaurant,Event Service,Falafel Restaurant,Factory,Eye Doctor,Exhibit,Event Space,Ethiopian Restaurant
632,New York,Williamsburg,40.707144,-73.958115,2.0,Grocery Store,Bar,Bagel Shop,Coffee Shop,Burger Joint,Tapas Restaurant,Vegetarian / Vegan Restaurant,Taco Place,Pizza Place,Clothing Store
278,London,Leytonstone,51.571078,0.006424,2.0,Pub,Café,Coffee Shop,Music Venue,Pharmacy,Grocery Store,Clothing Store,Bakery,Vietnamese Restaurant,Fast Food Restaurant
255,London,Kensington,51.498995,-0.199123,2.0,Café,Grocery Store,Sandwich Place,Italian Restaurant,Pub,Supermarket,Thai Restaurant,Bakery,Pharmacy,Burger Joint
243,London,Hornsey,51.587364,-0.120967,2.0,Pub,Supermarket,Pizza Place,Thai Restaurant,Park,Mediterranean Restaurant,Hotel,Farmers Market,Bakery,Acupuncturist


### 12. Sample Use Case <a name="12"></a>

For example, from the above data, if an employee gets transferred from Berkershiem neighborhood of Frankfurt to London and is looking for a similar neighborhood, Woodside Park neighborhood of London would be a reasonable choice, because both the neighborhoods belong to cluster 2.